In [1]:
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
import geopandas as gpd
from shapely.geometry import Polygon, MultiPoint , Point
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install keras


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
import math, time

from keras.models import model_from_json

In [62]:
qts2=pd.read_excel('/Users/erensmacbook/Downloads/2qts.xlsx')
qts5=pd.read_excel('/Users/erensmacbook/Downloads/5qts.xlsx')

qts2_t=qts2[['PERSID','AGEGROUP','SEX','WORKSTATUS','INDUSTRY']]
qts2_t=qts2_t[(qts2_t.INDUSTRY.isna() != True)] ## Ogrenci bilgileri icin mainact ve studying later

qts5_t=qts5[['PERSID','TRIPID',
      'ORIGPLACE','ORIGSA1_2021'
     ,'DESTPURP','DESTSA1_2021'
     ,'CUMDIST']]

real_data= pd.merge(qts2_t,qts5_t, on='PERSID')
real_data

# Set filepath
fp = "/Users/erensmacbook/Downloads/SA1_2021_AUST_SHP_GDA2020/SA1_2021_AUST_GDA2020.shp"

# Read file using gpd.read_file()
data = gpd.read_file(fp)

data1= data[(data.SA4_NAME21=='Brisbane - East') |
   (data.SA4_NAME21=='Brisbane - North')
    |
   (data.SA4_NAME21=='Brisbane - West')
    |
   (data.SA4_NAME21=='Brisbane Inner City')
           |
   (data.SA4_NAME21=='Brisbane - South')]

qld_orig = data1[['SA1_CODE21','geometry']]
qld_dest = data1[['SA1_CODE21','geometry']]

qld_orig=qld_orig.rename(columns={"SA1_CODE21":"ORIGSA1_2021"})
qld_dest=qld_dest.rename(columns={"SA1_CODE21":"DESTSA1_2021"})

qld_orig['ORIGSA1_2021']=qld_orig['ORIGSA1_2021'].astype(int)
qld_dest['DESTSA1_2021']=qld_dest['DESTSA1_2021'].astype(int)

qld_orig=qld_orig.rename(columns={"geometry":"geo_o"})
qld_dest=qld_dest.rename(columns={"geometry":"geo_d"})


train_data = pd.merge(real_data,qld_orig, on="ORIGSA1_2021")
train_data= pd.merge(train_data,qld_dest, on="DESTSA1_2021")



In [365]:
cop=train_data.copy()
act_ind=cop.PERSID.unique()

group_traject=cop.groupby('PERSID')
#group_traject.get_group(act_ind[2])

a_of_a=[]
industries=[]
string_list=[]
for i in act_ind:
    temp=group_traject.get_group(i)
    templist=temp.ORIGSA1_2021.to_list()
    
    son=temp.DESTSA1_2021.to_list()[-1]
    verdict=temp.INDUSTRY.values[0]
    
    templist.append(son)
    
    industries.append(verdict)
    templist=list(map(str, templist))
    vstr= ' '.join(templist)
    string_list.append(vstr)
    a_of_a.append(templist)

act_df=pd.DataFrame.from_records(a_of_a)

train_short=act_df[[0,1,2,3]]
train_short=train_short.fillna(0)
cols=list(map(str, range(0,4)))
train_short.columns = cols

In [1452]:
#b=cop.groupby(['PERSID'])
#b.size().value_counts()

In [228]:
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot


data = [[ [i+j] for i in range(5)] for j in range(100)]
target = [(i+5) for i in range(100)]

data_train =  np.array(data[0:80],dtype=float)
target_train = np.array(target[0:80],dtype=float)

data_test =  np.array(data[80:100],dtype=float)
target_test = np.array(target[80:100],dtype=float)


model = Sequential()

#model.add(Embedding(1,5,input_length=5 ) )
#model.add(LSTM((1), input_shape=(5,1),return_sequences=True))
#model.add(LSTM((1), input_shape=data_train.shape[1:],return_sequences=True))
model.add(LSTM((1), input_shape=(data_train.shape[1],1),return_sequences=True))


#model.add(Dense(3))
model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])



model.fit(data_train, target_train, epochs =2, batch_size=1, verbose=2,validation_data=(data_test, target_test))
predict = model.predict(data_test)

Epoch 1/2
80/80 - 2s - loss: 44.5164 - accuracy: 0.0000e+00 - val_loss: 94.5000 - val_accuracy: 0.0000e+00 - 2s/epoch - 26ms/step
Epoch 2/2
80/80 - 0s - loss: 44.5135 - accuracy: 0.0000e+00 - val_loss: 94.5000 - val_accuracy: 0.0000e+00 - 178ms/epoch - 2ms/step
1/1 [==============================] - 0s 393ms/step


In [ ]:
x_e = np.array([[0,1,2],[2,3,4]])
y_e = np.array([2,4])
y_e= to_categorical(y_e)


model3 = Sequential()
model3.add(Embedding(5, 20, input_length=3))
model3.add(LSTM(50))
model3.add(Dense(5,activation='softmax'))
model3.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

model3.fit(x_e,y_e, epochs =2)
#model3.predict(x_e,y_e)

In [893]:
flat_list = [item for sublist in a_of_a for item in sublist]

tokenizer=Tokenizer()
data= ' '.join(flat_list)
tokenizer.fit_on_texts([data])
encoded_data = tokenizer.texts_to_sequences([data])
#encoded_cod = pad_sequences(encoded_data,padding='post',truncating='post',maxlen=5)

In [915]:
string_list

['30504113303 30404110007 30404110331 30404110309 30404110303 30202103225 30202103205 30404110303 30202103205',
 '30501110519 30404110331 30501110519',
 '30501110532 30501110513 30404110331 30501110532 30501110513',
 '30501110901 30404110331 30501110901',
 '30501111204 30501110535 30501110541 30501110535 30501110531 30404110331 30501110535',
 '30501111035 30404110331 30501111035',
 '30503112614 30504113413 30404110331 30404110331 30504113413',
 '30203103909 30203103603 30404110331 30203103603',
 '30203103909 30203103909 30203103603 30404110331 30404110331 30203103603',
 '30501111001 30404110309 30404110309 30404110331 30404110331 30404110331 30404110309',
 '30501111001 30202102914 30404110331 30202102914 30202102802 30404110331 30202102914',
 '30404110007 30404110301 30404110331 30404110331 30404110301',
 '30404110007 30404110120 30404110331 30404110331 30404110120 30404110120',
 '30404110301 30404110331 30404110301',
 '30404110301 30403109713 30404110331 30403109713',
 '30404110301 30

In [916]:
' '.join(string_list)

'30504113303 30404110007 30404110331 30404110309 30404110303 30202103225 30202103205 30404110303 30202103205 30501110519 30404110331 30501110519 30501110532 30501110513 30404110331 30501110532 30501110513 30501110901 30404110331 30501110901 30501111204 30501110535 30501110541 30501110535 30501110531 30404110331 30501110535 30501111035 30404110331 30501111035 30503112614 30504113413 30404110331 30404110331 30504113413 30203103909 30203103603 30404110331 30203103603 30203103909 30203103909 30203103603 30404110331 30404110331 30203103603 30501111001 30404110309 30404110309 30404110331 30404110331 30404110331 30404110309 30501111001 30202102914 30404110331 30202102914 30202102802 30404110331 30202102914 30404110007 30404110301 30404110331 30404110331 30404110301 30404110007 30404110120 30404110331 30404110331 30404110120 30404110120 30404110301 30404110331 30404110301 30404110301 30403109713 30404110331 30403109713 30404110301 30404110331 30404110309 30404110301 30404110301 30404110323 304

In [897]:
vocab_size = len(tokenizer.word_index) +1
vocab_size

1614

In [930]:
sequences = []

#for k in encoded_data:
for i in range(1,len(encoded_data[0])):  #was k  # to pad 1-1 , for another value (3,len(encoded_data)) i-3 i+3
    sequence = encoded_data[0][i-1:i+1]
    sequences.append(sequence)

In [931]:
#sequences=np.array(sequences)
sequences

[[1505, 422],
 [422, 63],
 [63, 324],
 [324, 124],
 [124, 48],
 [48, 908],
 [908, 124],
 [124, 908],
 [908, 7],
 [7, 63],
 [63, 7],
 [7, 201],
 [201, 16],
 [16, 63],
 [63, 201],
 [201, 16],
 [16, 909],
 [909, 63],
 [63, 909],
 [909, 18],
 [18, 555],
 [555, 12],
 [12, 555],
 [555, 1],
 [1, 63],
 [63, 555],
 [555, 556],
 [556, 63],
 [63, 556],
 [556, 21],
 [21, 1063],
 [1063, 63],
 [63, 63],
 [63, 1063],
 [1063, 268],
 [268, 3],
 [3, 63],
 [63, 3],
 [3, 268],
 [268, 268],
 [268, 3],
 [3, 63],
 [63, 63],
 [63, 3],
 [3, 13],
 [13, 324],
 [324, 324],
 [324, 63],
 [63, 63],
 [63, 63],
 [63, 324],
 [324, 13],
 [13, 423],
 [423, 63],
 [63, 423],
 [423, 595],
 [595, 63],
 [63, 423],
 [423, 422],
 [422, 348],
 [348, 63],
 [63, 63],
 [63, 348],
 [348, 422],
 [422, 738],
 [738, 63],
 [63, 63],
 [63, 738],
 [738, 738],
 [738, 348],
 [348, 63],
 [63, 348],
 [348, 348],
 [348, 910],
 [910, 63],
 [63, 910],
 [910, 348],
 [348, 63],
 [63, 324],
 [324, 348],
 [348, 348],
 [348, 822],
 [822, 822],
 [822,

In [903]:
x=[]
y=[]
for i in sequences:
    x.append(i[0])
    y.append(i[1])



In [906]:
x = np.array(x)
y= np.array(y)

In [907]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y)

In [1459]:
x

array([1505,  422,   63, ...,  181, 1612, 1613])

In [1460]:
##1x1 but, orders not padded. not good.

from tensorflow import keras


model2 = Sequential()

#model2.add(Input(1,))
model2.add(Embedding(vocab_size, 20, input_length=1))
model2.add(LSTM(32,return_sequences=True))
model2.add(LSTM(32))

#model2.add(Dropout(0.5))
#model2.add(AttentionWeightedAverage())

model2.add(Dense(vocab_size,activation='softmax'))

#loss_fn = keras.losses.SparseCategoricalCrossentropy()

model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model2.fit(x,y, epochs =45)

#model = Sequential()
#model.add(Embedding(1,5,input_length=5 ) )
#model.add(LSTM((1), input_shape=(5,1),return_sequences=True))
#model.add(LSTM((1), input_shape=data_train.shape[1:],return_sequences=True))
#model.add(LSTM((1), input_shape=(data_train.shape[1],1),return_sequences=True))

Epoch 1/45
692/692 [==============================] - 7s 4ms/step - loss: 6.9039 - accuracy: 0.0150
Epoch 2/45
692/692 [==============================] - 2s 4ms/step - loss: 6.6890 - accuracy: 0.0160
Epoch 3/45
692/692 [==============================] - 3s 4ms/step - loss: 6.6657 - accuracy: 0.0161
Epoch 4/45
692/692 [==============================] - 2s 4ms/step - loss: 6.6497 - accuracy: 0.0172
Epoch 5/45
692/692 [==============================] - 2s 4ms/step - loss: 6.6225 - accuracy: 0.0199
Epoch 6/45
692/692 [==============================] - 2s 4ms/step - loss: 6.5434 - accuracy: 0.0219
Epoch 7/45
692/692 [==============================] - 2s 4ms/step - loss: 6.4302 - accuracy: 0.0229
Epoch 8/45
692/692 [==============================] - 2s 4ms/step - loss: 6.3481 - accuracy: 0.0231
Epoch 9/45
692/692 [==============================] - 2s 4ms/step - loss: 6.2843 - accuracy: 0.0241
Epoch 10/45
692/692 [==============================] - 2s 4ms/step - loss: 6.2294 - accuracy: 0.0249

KeyboardInterrupt: 

In [1232]:
em=[]
from keras.layers import Bidirectional,SpatialDropout1D


def build_model(maxlen, vocab_size, embedding_size, embedding_matrix):
    input_words = Input((maxlen, ))
    x_words = (Embedding(vocab_size, 20, input_length=1))(input_words)
    #x_words = SpatialDropout1D(0.3)(x_words)
    x_words = Bidirectional(LSTM(64, return_sequences=True))(x_words)
    x_words = Bidirectional(LSTM(64, return_sequences=True))(x_words)

    x = Attention(maxlen)(x_words)
    #x = Dropout(0.2)(x)
    #x = Dense(50, activation='relu')(x)
    #x = Dropout(0.2)(x)
    pred = Dense(vocab_size, activation='softmax')(x)

    model = Model(inputs=input_words, outputs=pred)
    return model

model = build_model(1, vocab_size,5, em)
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

model.fit(x,y, epochs=100, verbose=1)

Epoch 1/100
692/692 [==============================] - 13s 7ms/step - loss: 6.8648 - accuracy: 0.0152
Epoch 2/100
692/692 [==============================] - 5s 7ms/step - loss: 6.6958 - accuracy: 0.0162
Epoch 3/100
692/692 [==============================] - 5s 7ms/step - loss: 6.6690 - accuracy: 0.0154
Epoch 4/100
692/692 [==============================] - 5s 7ms/step - loss: 6.5755 - accuracy: 0.0204
Epoch 5/100
692/692 [==============================] - 5s 7ms/step - loss: 6.4093 - accuracy: 0.0225
Epoch 6/100
692/692 [==============================] - 5s 7ms/step - loss: 6.3113 - accuracy: 0.0243
Epoch 7/100
692/692 [==============================] - 5s 7ms/step - loss: 6.2308 - accuracy: 0.0244
Epoch 8/100
692/692 [==============================] - 5s 7ms/step - loss: 6.1601 - accuracy: 0.0259
Epoch 9/100
692/692 [==============================] - 5s 7ms/step - loss: 6.0853 - accuracy: 0.0281
Epoch 10/100
692/692 [==============================] - 5s 7ms/step - loss: 5.9998 - accur

692/692 [==============================] - 5s 7ms/step - loss: 3.7767 - accuracy: 0.1565
Epoch 82/100
692/692 [==============================] - 5s 7ms/step - loss: 3.7660 - accuracy: 0.1575
Epoch 83/100
692/692 [==============================] - 6s 9ms/step - loss: 3.7601 - accuracy: 0.1563
Epoch 84/100
692/692 [==============================] - 6s 8ms/step - loss: 3.7522 - accuracy: 0.1572
Epoch 85/100
692/692 [==============================] - 5s 7ms/step - loss: 3.7418 - accuracy: 0.1575
Epoch 86/100
692/692 [==============================] - 5s 7ms/step - loss: 3.7355 - accuracy: 0.1564
Epoch 87/100
692/692 [==============================] - 6s 9ms/step - loss: 3.7284 - accuracy: 0.1560
Epoch 88/100
692/692 [==============================] - 5s 7ms/step - loss: 3.7213 - accuracy: 0.1575
Epoch 89/100
692/692 [==============================] - 5s 7ms/step - loss: 3.7107 - accuracy: 0.1570
Epoch 90/100
692/692 [==============================] - 5s 8ms/step - loss: 3.7060 - accuracy: 

In [1261]:
y_pred = model2.predict(x)
y_pred = y_pred.argmax(axis=1).astype(int)
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
words = [reverse_word_map.get(letter) for letter in y_pred]
firsts = [reverse_word_map.get(letter) for letter in x]


692/692 [==============================] - 1s 2ms/step


In [1278]:
firsts[5]

'30202103225'

In [1279]:
words[5]

'30202102820'

In [1135]:
model1 = Sequential()
model1.add(Embedding(20, 10, trainable=True))
model1.add(Dense(1, activation='sigmoid'))
model2 = Sequential()
model2.add(Embedding(20, 10, trainable=True))
model2.add(Dense(1, activation='sigmoid'))

model_concat = concatenate([model1.output, model2.output], axis=-1)

In [1131]:
from keras.layers import Reshape, Input
from keras.models import Model


model1_in = Input(shape=(27, 27, 1))
model1_out = Dense(300, input_dim=40, activation='relu')(model1_in)
model1 = Model(model1_in, model1_out)

model2_in = Input(shape=(27, 27, 1))
model2_out = Dense(300, input_dim=40, activation='relu')(model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])


In [1140]:
concatenated = concatenate([model1_in, model1_in])


In [2244]:
encodeea = np.array([2,3])
pad_sequences([encodeea], maxlen=10)

array([[0, 0, 0, 0, 0, 0, 0, 0, 2, 3]], dtype=int32)

In [1891]:
'''labels= np.array([1,1,1,1,1,0,0,0,0,0])

model3e = Sequential()
model3e.add(Embedding(1614, 20, input_length=4))
model3e.add(LSTM(50))
model3e.add(Dense(1614,activation='softmax'))
model3e.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model3e.fit(encoded_cod[0:10],y, epochs =2)
#model3.predict(x_e,y_e)'''

"labels= np.array([1,1,1,1,1,0,0,0,0,0])\n\nmodel3e = Sequential()\nmodel3e.add(Embedding(1614, 20, input_length=4))\nmodel3e.add(LSTM(50))\nmodel3e.add(Dense(1614,activation='softmax'))\nmodel3e.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])\n\nmodel3e.fit(encoded_cod[0:10],y, epochs =2)\n#model3.predict(x_e,y_e)"

In [ ]:
tokenizera=Tokenizer()
#data= ' '.join(flat_list)
tokenizera.fit_on_texts(a_of_a)
encoded_dataa = tokenizera.texts_to_sequences(a_of_a)
encoded_cod = pad_sequences(encoded_dataa,padding='post',truncating='post',maxlen=4)

In [1945]:
'''sequences_cod=[]

for i in range(0,len(encoded_cod)):
    for k in range(2, len(encoded_cod[i])):
        sequence_cod = encoded_cod[i][k-2:k+1]
        sequences_cod.append(sequence_cod)
    ''' ## already sequenced

x_cod=[]
y_cod=[]
for i in encoded_cod:
    x_cod.append(i[0:3])
    #y_cod.append(i[1:4])
    y_cod.append(i[3])


x_cod = np.array(x_cod)
y_cod= np.array(y_cod)
    
y_cod = to_categorical(y_cod, num_classes=vocab_size)

        

In [2251]:
import tensorflow as tf

In [2252]:
#from tensorflow.keras.callbacks import LambdaCallback
#print_lambda_callback = LambdaCallback()

#model nx1
keras.backend.clear_session()
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

model3ecod = Sequential()
#model3ecod.add(Input(3,))
model3ecod.add(Embedding(1614, 20, input_length=3))
#model3ecod.add(Dropout(0.2))
model3ecod.add(LSTM(50,return_sequences=True))
model3ecod.add(LSTM(50))
#model3ecod.add(Dropout(0.2))
#model3ecod.add(LSTM(50, return_sequences=True))
model3ecod.add(Dense(1614,activation='sigmoid'))
model3ecod.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])




model3ecod.fit(x_cod,y_cod, epochs =20,batch_size=10,callbacks=[print_lambda_callback])
#model3.predict(x_e,y_e)

Epoch 1/20
536/536 [==============================] - 25s 47ms/step - loss: 4.2407 - accuracy: 0.5551
Epoch 2/20
536/536 [==============================] - 25s 47ms/step - loss: 3.4861 - accuracy: 0.5564
Epoch 3/20
536/536 [==============================] - 25s 47ms/step - loss: 3.4067 - accuracy: 0.5564
Epoch 4/20
536/536 [==============================] - 25s 48ms/step - loss: 3.3615 - accuracy: 0.5564
Epoch 5/20
536/536 [==============================] - 25s 47ms/step - loss: 3.3182 - accuracy: 0.5564
Epoch 6/20
536/536 [==============================] - 25s 47ms/step - loss: 3.2494 - accuracy: 0.5564
Epoch 7/20
536/536 [==============================] - 25s 47ms/step - loss: 3.1690 - accuracy: 0.5564
Epoch 8/20
536/536 [==============================] - 27s 51ms/step - loss: 3.0913 - accuracy: 0.5564
Epoch 9/20
536/536 [==============================] - 26s 48ms/step - loss: 3.0164 - accuracy: 0.5568
Epoch 10/20
536/536 [==============================] - 26s 48ms/step - loss: 2.943

In [2260]:
np.argsort(model3ecod.predict( np.array([[1505,22,34]]) ))[0][-21:]
#np.argsort(model3ecod.predict( x_cod[1:2] ))[0][-21:]


1/1 [==============================] - 0s 38ms/step


array([120, 348, 150,  78, 477,  10, 547, 526, 158, 230,   0, 117, 129,
        76,  31, 176,  20, 200, 152, 174, 195])

In [2264]:
np.argsort(model2.predict( np.array([[1505,22,22,22]]) ))[0][-21:]


1/1 [==============================] - 0s 40ms/step


array([ 21, 301, 218, 282, 243, 254,  70,  36, 195, 544,  11, 193,  49,
         8,  44, 105,  39,   9,  22,   2,   0])

In [ ]:
### model n to n

In [1540]:
model3ecod = Sequential()
model3ecod.add(Embedding(1614, 20, input_length=3))
#model3ecod.add(Dropout(0.2))
model3ecod.add(LSTM(50,return_sequences=True))
model3ecod.add(LSTM(50,return_sequences=True))
#model3ecod.add(Dropout(0.2))
#model3ecod.add(LSTM(50, return_sequences=True))
model3ecod.add(Dense(1614,activation='sigmoid'))
model3ecod.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model3ecod.fit(x_cod,y_cod, epochs =150,batch_size=10)
#model3.predict(x_e,y_e)

Epoch 1/150
536/536 [==============================] - 6s 5ms/step - loss: 5.9966 - accuracy: 0.2120
Epoch 2/150
536/536 [==============================] - 3s 5ms/step - loss: 5.5566 - accuracy: 0.2127
Epoch 3/150
536/536 [==============================] - 3s 5ms/step - loss: 5.4603 - accuracy: 0.2208
Epoch 4/150
536/536 [==============================] - 3s 5ms/step - loss: 5.4037 - accuracy: 0.2262
Epoch 5/150
536/536 [==============================] - 3s 5ms/step - loss: 5.3495 - accuracy: 0.2262
Epoch 6/150
536/536 [==============================] - 3s 5ms/step - loss: 5.2833 - accuracy: 0.2262
Epoch 7/150
536/536 [==============================] - 3s 5ms/step - loss: 5.1964 - accuracy: 0.2262
Epoch 8/150
536/536 [==============================] - 3s 5ms/step - loss: 5.0963 - accuracy: 0.2255
Epoch 9/150
536/536 [==============================] - 3s 5ms/step - loss: 4.9928 - accuracy: 0.2258
Epoch 10/150
536/536 [==============================] - 3s 5ms/step - loss: 4.8847 - accura

536/536 [==============================] - 3s 5ms/step - loss: 1.7032 - accuracy: 0.6372
Epoch 82/150
536/536 [==============================] - 2s 5ms/step - loss: 1.6865 - accuracy: 0.6369
Epoch 83/150
536/536 [==============================] - 2s 5ms/step - loss: 1.6710 - accuracy: 0.6407
Epoch 84/150
536/536 [==============================] - 3s 5ms/step - loss: 1.6582 - accuracy: 0.6410
Epoch 85/150
536/536 [==============================] - 3s 5ms/step - loss: 1.6420 - accuracy: 0.6452
Epoch 86/150
536/536 [==============================] - 3s 5ms/step - loss: 1.6290 - accuracy: 0.6468
Epoch 87/150
536/536 [==============================] - 3s 5ms/step - loss: 1.6162 - accuracy: 0.6487
Epoch 88/150
536/536 [==============================] - 2s 5ms/step - loss: 1.6023 - accuracy: 0.6506
Epoch 89/150
536/536 [==============================] - 3s 5ms/step - loss: 1.5873 - accuracy: 0.6511
Epoch 90/150
536/536 [==============================] - 3s 5ms/step - loss: 1.5710 - accuracy: 

In [2132]:
model3ecod.summary()

Model: "sequential_276"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_289 (Embedding)   (None, 3, 20)             32280     
                                                                 
 lstm_311 (LSTM)             (None, 3, 50)             14200     
                                                                 
 lstm_312 (LSTM)             (None, 50)                20200     
                                                                 
 dense_240 (Dense)           (None, 1614)              82314     
                                                                 
Total params: 148,994
Trainable params: 148,994
Non-trainable params: 0
_________________________________________________________________


In [2129]:
model2.summary()

Model: "sequential_272"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_285 (Embedding)   (None, 1, 20)             32280     
                                                                 
 lstm_303 (LSTM)             (None, 1, 50)             14200     
                                                                 
 lstm_304 (LSTM)             (None, 50)                20200     
                                                                 
 dense_236 (Dense)           (None, 1614)              82314     
                                                                 
Total params: 148,994
Trainable params: 148,994
Non-trainable params: 0
_________________________________________________________________


In [1595]:
### 1n1 models

In [1604]:
x_1n1=[]
y_1n1=[]
for i in encoded_cod:
    for k in range(1,len(i)): 
    #for k in i:
        #print(i)
        #print(k)
        #sequence = i[k-1:k+1]
        x_1n1.append(i[k-1])
        y_1n1.append(i[k])

x_1n1 = np.array(x_1n1)
y_1n1= np.array(y_1n1)        
y_1n1 = to_categorical(y_1n1,vocab_size)

In [1610]:

model2 = Sequential()

#model2.add(Input(1,))
model2.add(Embedding(vocab_size, 20, input_length=1))
model2.add(LSTM(50,return_sequences=True))
model2.add(LSTM(50,return_sequences=False))

#model2.add(Dropout(0.5))
#model2.add(AttentionWeightedAverage())

model2.add(Dense(vocab_size,activation='softmax'))

#loss_fn = keras.losses.SparseCategoricalCrossentropy()

model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model2.fit(x_1n1,y_1n1, epochs =100)

Epoch 1/100
503/503 [==============================] - 6s 5ms/step - loss: 6.1298 - accuracy: 0.2122
Epoch 2/100
503/503 [==============================] - 2s 5ms/step - loss: 5.6697 - accuracy: 0.2127
Epoch 3/100
503/503 [==============================] - 3s 5ms/step - loss: 5.6495 - accuracy: 0.2127
Epoch 4/100
503/503 [==============================] - 4s 8ms/step - loss: 5.6398 - accuracy: 0.2127
Epoch 5/100
503/503 [==============================] - 3s 6ms/step - loss: 5.6351 - accuracy: 0.2127
Epoch 6/100
503/503 [==============================] - 2s 5ms/step - loss: 5.6288 - accuracy: 0.2127
Epoch 7/100
503/503 [==============================] - 2s 5ms/step - loss: 5.6259 - accuracy: 0.2127
Epoch 8/100
503/503 [==============================] - 2s 4ms/step - loss: 5.6213 - accuracy: 0.2127
Epoch 9/100
503/503 [==============================] - 3s 5ms/step - loss: 5.6184 - accuracy: 0.2127
Epoch 10/100
503/503 [==============================] - 3s 5ms/step - loss: 5.6151 - accura

503/503 [==============================] - 2s 5ms/step - loss: 3.9340 - accuracy: 0.2450
Epoch 82/100
503/503 [==============================] - 3s 5ms/step - loss: 3.9162 - accuracy: 0.2470
Epoch 83/100
503/503 [==============================] - 3s 5ms/step - loss: 3.9011 - accuracy: 0.2456
Epoch 84/100
503/503 [==============================] - 2s 5ms/step - loss: 3.8850 - accuracy: 0.2465
Epoch 85/100
503/503 [==============================] - 2s 4ms/step - loss: 3.8698 - accuracy: 0.2471
Epoch 86/100
503/503 [==============================] - 2s 4ms/step - loss: 3.8544 - accuracy: 0.2483
Epoch 87/100
503/503 [==============================] - 2s 4ms/step - loss: 3.8403 - accuracy: 0.2490
Epoch 88/100
503/503 [==============================] - 2s 4ms/step - loss: 3.8260 - accuracy: 0.2479
Epoch 89/100
503/503 [==============================] - 2s 4ms/step - loss: 3.8113 - accuracy: 0.2483
Epoch 90/100
503/503 [==============================] - 2s 4ms/step - loss: 3.7981 - accuracy: 

In [1890]:
def generate_seq(model,tokenizer,enter_text,n_pred):
    
    in_text,result= enter_text,enter_text
    
    for _ in range(n_pred):
        
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = np.array(encoded) 
    
        #yhat= model.predict_classes(encoded)
        predict_x=model.predict(encoded[0:1]) 
        yhat=np.argmax(predict_x,axis=1)
        
        out_word=''
        
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
            in_text, result = out_word, result + ' ' + out_word
            
    return result

In [1888]:
def pred_sample(x):
    preds= model2.predict(x,batch_size=1)
    #tahmin=np.argmax(aaaa,axis=1)
    #aaaa2=model2.predict(tahmin)
    #np.argmax(aaaa2,axis=1)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 2.220446049250313e-16)/0.7 # temperature #K.epsilon()
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    probas = np.random.multinomial(1,preds[0],1)
    index = np.argmax(probas)
    
    if index == 0:
        index = np.argsort(preds)[0][-2]
        
        
    elif index == x[0]:                           #new block, what if not 0 but itself, it will repeat only same....
        index = np.argsort(preds)[0][-2]
        
        
        
    else:
        # return list of top N chars/words
        # descending order, based on probability
        index = (-preds)[0].argsort()[:3]
        
    return index

In [1889]:
#pred_sample(x_1n1[0:1])
pred_sample([2])

1/1 [==============================] - 0s 41ms/step


array([0, 2, 4])

In [1916]:
len([2])

1

In [28]:
def build_model(layers):
    model = Sequential()

    for x in range(0,3):
        model.add(LSTM(input_dim=layers[0], units=layers[1], return_sequences=True))
        model.add(Dropout(0.1))

    model.add(LSTM(layers[2], return_sequences=False)) 
    model.add(Dropout(0.1))

    model.add(Dense(units=layers[2]))
    model.add(Activation("tanh"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

In [32]:
x_train = x[0:15000]
y_train = y[0:15000]
x_test = x[15000:-1]
y_test = y[15000:-1]

In [54]:
def max_value(inputlist):
    return max([sublist[-1] for sublist in inputlist])

print(max_value(y_train))

1571


In [55]:
seq_len = 4 # Another hyperparameter- length of hurricane?
feature_count = 1577
model = build_model([feature_count, seq_len, 1])

Compilation Time :  0.00391387939453125


In [56]:
model.fit(x_train, y_train, batch_size=512, epochs=10, validation_split=0.1, verbose=1)


ValueError: `validation_split` is only supported for Tensors or NumPy arrays, found following types in the input: [<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, 

In [380]:
x_train.shape
#y_train.shape

(50000, 1, 5)

/var/folders/jv/2_fp3b717yl0vd8xbytqhz300000gn/T/ipykernel_92982/1433173024.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[i] = codes


In [491]:
trainScore = model.evaluate(x_train, y_train, verbose=0)


In [492]:
print('Train Score: %.4f MSE (%.4f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))


Train Score: 7567847.0000 MSE (2750.9720 RMSE)


In [485]:
testScore = model.evaluate(x_test, y_test, verbose=0)


In [486]:
print('Train Score: %.4f MSE (%.4f RMSE)' % (testScore[0], math.sqrt(testScore[0])))


Train Score: 7937603.0000 MSE (2817.3752 RMSE)


In [527]:
pred = model.predict(x_test)


In [528]:
pred

array([[0.98367023, 0.9943218 , 0.998887  ],
       [0.98367023, 0.9943218 , 0.998887  ],
       [0.98367023, 0.9943218 , 0.998887  ],
       ...,
       [0.98367023, 0.9943218 , 0.998887  ],
       [0.98367023, 0.9943218 , 0.998887  ],
       [0.98367023, 0.9943218 , 0.998887  ]], dtype=float32)

In [404]:
y_test

array([1699, 1669, 1377, ...,  758,  339,  399])

In [518]:
x_train

array([[[   9,    0,    4, 4268]],

       [[   9,    0,    4, 4329]],

       [[   9,    1,    4, 4268]],

       ...,

       [[   7,    0,    7,  127]],

       [[   7,    1,    4,  177]],

       [[   7,    1,    4, 1063]]])

In [1146]:
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import backend as K
from tensorflow.keras import initializers

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for
    a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 trainable=True,
                                 initializer=self.init)
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0],
                                                   input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [1200]:
from keras import initializers, regularizers, constraints

class Attention(Layer):
    """
    Keras Layer that implements an Attention mechanism for temporal data.
    Supports Masking.
    Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    :param kwargs:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(Attention())
    """
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        
            
        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None
        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
        if self.bias:
            eij += self.b
        eij = K.tanh(eij)
        a = K.exp(eij)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim